In [1]:
import gradio as gr
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from sentence_transformers import SentenceTransformer

nltk.download("punkt", quiet=True)
nltk.download("stopwords", quiet=True)
nltk.download('punkt_tab', quiet=True)
model = SentenceTransformer("distiluse-base-multilingual-cased-v2")

def summarize(text: str, k: int):
    if not text.strip():
        return "", 0, 0, 0.0

    sentences = sent_tokenize(text)
    k = min(k, len(sentences))

    stop_words = set(stopwords.words("english"))
    def remove_sw(s: str) -> str:
        return " ".join(w for w in s.lower().split() if w not in stop_words)
    cleaned = [remove_sw(s) for s in sentences]

    embeds = model.encode([s.lower() for s in sentences])

    n = len(sentences)
    sim = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                sim[i, j] = cosine_similarity(
                    embeds[i].reshape(1, -1), embeds[j].reshape(1, -1)
                )[0, 0]

    graph = nx.from_numpy_array(sim)
    try:
        ranks = nx.pagerank(graph, max_iter=1000, tol=1e-06)
    except nx.PowerIterationFailedConvergence:
        return "PageRank failed to converge. Try shorter text.", 0, 0, 0.0

    ranked = sorted(((ranks[i], s, i) for i, s in enumerate(sentences)),
                    reverse=True)[:k]
    summary = " ".join(s for _, s, _ in sorted(ranked, key=lambda x: x[2]))

    orig_words = len(text.split())
    summ_words = len(summary.split())
    reduction = (1 - summ_words / orig_words) * 100 if orig_words else 0

    return summary, orig_words, summ_words, reduction

def update_slider(text):
    sentences = sent_tokenize(text)
    count = len(sentences)

    if count < 2:
        # Return a dummy range that avoids math error
        return gr.update(minimum=1, maximum=2, value=1, interactive=False)

    return gr.update(minimum=1, maximum=count, value=min(3, count), interactive=True)


with gr.Blocks(title="Text Summarizer") as demo:
    gr.Markdown("# 🧠 Text Summarizer (English)")
    gr.Markdown(
        "Paste an English article or text below, select how many sentences "
        "you want in the summary, and click **Summarize**."
    )

    with gr.Row():
        text_input = gr.Textbox(lines=12, label="Enter your text here:")
        slider = gr.Slider(minimum=1, maximum=2, value=1, label="Summary Sentence Count", interactive=False)


    text_input.change(fn=update_slider, inputs=text_input, outputs=slider)

    summarize_btn = gr.Button("Summarize")
    output_summary = gr.Textbox(label="📝 Summary", lines=5)
    word_stats = gr.Markdown()

    def run_summary(text, k):
        summary, orig_len, summ_len, reduction = summarize(text, k)
        stats_md = (
            f"**📊 Summary Stats**\n\n"
            f"- Original Length: `{orig_len} words`\n"
            f"- Summary Length: `{summ_len} words`\n"
            f"- Reduction: `{reduction:.1f}%`"
        )
        return summary, stats_md

    summarize_btn.click(
        fn=run_summary,
        inputs=[text_input, slider],
        outputs=[output_summary, word_stats]
    )

demo.launch()


c:\Users\V16RKhalil\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\V16RKhalil\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\gradio\queueing.py", line 626, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\V16RKhalil\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\V16RKhalil\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\gradio\blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\V16RKhalil\AppData\Local\miniconda3\envs\myenv\Lib\site-packages\gradio\blocks.py", line 1731, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\V16RKhalil\AppDat